# LangChain Workshop: Building an AI-Powered Chatbot
### CCDS Tech for Good 2026 Hackathon

**Workshop Goals:**
- Understand LangChain fundamentals
- Build conversational AI agents
- Create production-ready chatbot classes
- Master prompt engineering techniques

**What you'll build:**
- Basic chatbot with memory
- AI companion with personality
- Tool-using agents
- Reusable chatbot framework

# 1. Environment Setup

## **uv : A Fast Python Management Setup**

**[Official Docs](https://docs.astral.sh/uv/getting-started/installation/)** `Click this link to install`

`uv` replaces standard tools (`pip`, `venv`, `virtualenv`) with a single, unified toolkit that is significantly faster.

---

### **Core Commands**

| Action | Command | Description |
| --- | --- | --- |
| **Add Library** | `uv add <package>` | Installs the package and adds it to `pyproject.toml`. |
| **View Graph** | `uv tree` | Visualizes the full dependency tree (parents & children). |
| **Restore** | `uv sync` | **The only command needed after cloning.** Installs everything from `uv.lock`. |
| **Run Script** | `uv run <script.py>` | Runs a script inside the managed environment automatically. |

---

### **Dependency Management Files**

* **`pyproject.toml` ( The "Wishlist" )**
* Lists the high-level packages you explicitly requested (e.g., `langchain`, `numpy`).
* This is the file you edit if you want to change project settings.


* **`uv.lock` ( The "Snapshot" )**
* Contains the **exact** version and hash of every single package installed (including transitive dependencies).
* Ensures that everyone working on the project has the exact same environment.



---

### **Generated File Hierarchy**

When you initialize and use `uv`, your project structure will look like this:

```text
my_project/
├── .venv/               # [Hidden] The actual virtual environment containing binaries/libs.
├── .python-version      # Pins the specific Python version (e.g., 3.12).
├── pyproject.toml       # User-defined dependencies.
├── uv.lock              # Machine-generated freeze of all dependencies.
└── src/                 # (Recommended) Your source code folder.
    └── main.py

```

---

### **Workflow for Cloning**

`uv sync`

This single command reads `uv.lock`, creates the `.venv`, and installs all locked dependencies to match the original environment perfectly.

## API Key Configuration

Create a `.env` file in your project directory:

```
AZURE_OPENAI_API_KEY=your_api_key_here
AZURE_OPENAI_ENDPOINT=your_endpoint_here
```

**⚠️ Important**: Never commit your `.env` file to GitHub! Add it to `.gitignore`

In [ ]:
# Verify installation
import sys

try:
    import langchain
    import langchain_openai
    from dotenv import load_dotenv
    print("All packages installed successfully!")
    print(f"Python version: {sys.version}")
    print(f"LangChain version: {langchain.__version__}")
except ImportError as e:
    print(f"Missing package: {e}")
    print("Run: pip install langchain langchain-openai python-dotenv")

# 2. LangChain Fundamentals

## 2.1 Your First LLM Call

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize the LLM
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-5-nano",
)

# Create messages
messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="Explain what LangChain is in one sentence.")
]

# Get response
response = llm.invoke(messages)
print(response.content)

###  Understanding the Components

**1. SystemMessage**: Defines the AI's role and behavior
- Sets personality and expertise
- Provides context and constraints
- Remains consistent across conversation

**2. HumanMessage**: User's input to the AI
- The query or prompt
- Changes with each interaction

**3. Temperature**: Controls randomness (0-1)
- `0.0` → Deterministic, focused
- `0.7` → Balanced (recommended)
- `1.0` → Creative, varied

## 2.2 Adding Conversation Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Initialize LLM
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-5-nano"
)

# Create memory buffer
memory = ConversationBufferMemory()

# Create conversation chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True  # Shows internal processing
)

# Multi-turn conversation
print("Turn 1:")
response1 = conversation.predict(input="Hi! My name is Sarah.")
print(f"AI: {response1}\n")

print("Turn 2:")
response2 = conversation.predict(input="What's my name?")  # AI remembers!
print(f"AI: {response2}\n")

print("Turn 3:")
response3 = conversation.predict(input="What did we just talk about?")
print(f"AI: {response3}")

## 2.3 Different Memory Types

In [ ]:
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory
)

# 1. Buffer Memory - Stores ALL conversation history
buffer_memory = ConversationBufferMemory()
buffer_memory.save_context({"input": "Hi!"}, {"output": "Hello!"})
buffer_memory.save_context({"input": "How are you?"}, {"output": "I'm doing well!"})
print("=== Buffer Memory (Stores Everything) ===")
print(buffer_memory.load_memory_variables({}))

print("\n" + "="*60 + "\n")

# 2. Window Memory - Only keeps last K interactions (RECOMMENDED)
window_memory = ConversationBufferWindowMemory(k=2)  # Last 2 exchanges only
window_memory.save_context({"input": "Message 1"}, {"output": "Response 1"})
window_memory.save_context({"input": "Message 2"}, {"output": "Response 2"})
window_memory.save_context({"input": "Message 3"}, {"output": "Response 3"})
print("=== Window Memory (k=2, Most Efficient) ===")
print(window_memory.load_memory_variables({}))
print("Note: Message 1 was forgotten!")

print("\n" + "="*60 + "\n")

# 3. Summary Memory - Summarizes old conversations
summary_memory = ConversationSummaryMemory(llm=llm)
summary_memory.save_context(
    {"input": "Tell me about machine learning"}, 
    {"output": "Machine learning is a subset of AI that enables computers to learn from data without explicit programming..."}
)
print("=== Summary Memory (Summarizes History) ===")
print(summary_memory.load_memory_variables({}))

print("\n💡 Recommendation: Use ConversationBufferWindowMemory for your projects!")

## 2.4 Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Create a flexible prompt template
template = """
You are a {personality} AI assistant specialized in {domain}.

User's question: {question}

Provide a {tone} response.
"""

prompt = PromptTemplate(
    input_variables=["personality", "domain", "question", "tone"],
    template=template
)

# Create chain
chain = LLMChain(llm=llm, prompt=prompt)

# Example 1: Friendly tutor
print("=== Friendly Tutor ===")
response = chain.predict(
    personality="friendly and encouraging",
    domain="mathematics",
    question="What is the Pythagorean theorem?",
    tone="simple and easy to understand"
)
print(response)

print("\n" + "="*60 + "\n")

# Example 2: Professional advisor
print("=== Professional Advisor ===")
response = chain.predict(
    personality="professional and analytical",
    domain="career development",
    question="How do I prepare for a data science interview?",
    tone="structured and actionable"
)
print(response)

# 3. Building Your AI Companion

## 3.1 Basic Chatbot with Personality

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

# Define chatbot personality
chatbot_personality = """
You are Alex, a friendly and knowledgeable AI companion. You are:
- Patient and understanding
- Enthusiastic about learning and helping
- Conversational and warm
- Clear and concise in explanations

Always respond in a friendly, natural way. Keep responses focused and helpful.
"""

# Create custom prompt
prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=f"{chatbot_personality}\n\nConversation history:\n{{history}}\n\nHuman: {{input}}\nAlex:"
)

# Initialize chatbot
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-35-turbo",
    temperature=0.7
)

memory = ConversationBufferWindowMemory(k=5)
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=False
)

# Test the chatbot
print("Testing Alex the AI Companion:\n")
print("You: Hi Alex! How are you today?")
print(f"Alex: {conversation.predict(input='Hi Alex! How are you today?')}\n")

print("You: Can you help me with my studies?")
print(f"Alex: {conversation.predict(input='Can you help me with my studies?')}\n")

print("You: What was the first thing I asked you?")
print(f"Alex: {conversation.predict(input='What was the first thing I asked you?')}")

## 3.2 Reusable Chatbot Class

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import os

class AICompanion:
    """
    A flexible AI chatbot class that can be customized for different purposes.
    """
    
    def __init__(self, name, personality, domain=None, memory_window=5):
        """
        Initialize the AI companion.
        
        Args:
            name (str): Name of the AI companion
            personality (str): Personality description
            domain (str, optional): Area of expertise
            memory_window (int): Number of exchanges to remember
        """
        self.name = name
        self.personality = personality
        self.domain = domain
        
        # Initialize LLM
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2024-02-15-preview",
            deployment_name="gpt-35-turbo",
            temperature=0.7
        )
        
        # Setup memory
        self.memory = ConversationBufferWindowMemory(k=memory_window)
        
        # Create conversation chain
        self._setup_conversation()
    
    def _setup_conversation(self):
        """Setup the conversation chain with custom prompt"""
        
        # Build system prompt
        system_prompt = f"You are {self.name}. {self.personality}"
        if self.domain:
            system_prompt += f"\n\nYou specialize in {self.domain}."
        
        system_prompt += "\n\nProvide helpful, clear, and friendly responses."
        
        # Create prompt template
        template = f"{system_prompt}\n\nConversation history:\n{{history}}\n\nHuman: {{input}}\n{self.name}:"
        
        prompt = PromptTemplate(
            input_variables=["history", "input"],
            template=template
        )
        
        # Create conversation chain
        self.conversation = ConversationChain(
            llm=self.llm,
            memory=self.memory,
            prompt=prompt,
            verbose=False
        )
    
    def chat(self, user_input):
        """
        Send a message to the AI companion.
        
        Args:
            user_input (str): User's message
        
        Returns:
            str: AI's response
        """
        try:
            response = self.conversation.predict(input=user_input)
            return response
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
    
    def reset_memory(self):
        """Clear conversation history"""
        self.memory.clear()
        print(f"{self.name}'s memory has been cleared!")
    
    def get_conversation_history(self):
        """Get the current conversation history"""
        return self.memory.load_memory_variables({})

# Example 1: Study Buddy
print("=== Example 1: Study Buddy ===")
study_buddy = AICompanion(
    name="StudyBot",
    personality="You are patient, encouraging, and great at explaining complex topics simply.",
    domain="mathematics and science"
)

print(f"User: Explain Newton's first law")
print(f"StudyBot: {study_buddy.chat('Explain Newton\'s first law')}\n")

# Example 2: Wellness Coach
print("\n=== Example 2: Wellness Coach ===")
wellness_coach = AICompanion(
    name="WellnessBot",
    personality="You are supportive, motivating, and focused on healthy habits.",
    domain="health and wellness"
)

print(f"User: Give me tips for better sleep")
print(f"WellnessBot: {wellness_coach.chat('Give me tips for better sleep')}")

## 3.3 Interactive Console Chatbot

In [ ]:
def run_interactive_chatbot():
    """
    Run an interactive chat session in the console.
    """
    print("="*60)
    print("Interactive AI Companion")
    print("Commands: 'quit' to exit, 'reset' to clear memory")
    print("="*60 + "\n")
    
    # Create chatbot
    bot = AICompanion(
        name="Companion",
        personality="You are friendly, helpful, and conversational."
    )
    
    print("Companion: Hi! I'm your AI companion. How can I help you today?\n")
    
    while True:
        # Get user input
        user_input = input("You: ").strip()
        
        # Check for commands
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("\nCompanion: Goodbye! Have a great day!")
            break
        
        if user_input.lower() == 'reset':
            bot.reset_memory()
            continue
        
        if not user_input:
            continue
        
        # Get and display response
        response = bot.chat(user_input)
        print(f"\nCompanion: {response}\n")

# Run the interactive chatbot
# Uncomment the line below to start chatting!
# run_interactive_chatbot()

# 4. Advanced: Agents with Tools

## 4.1 Understanding Agents

**Agents** are LLMs that can:
- Use external tools
- Make decisions about which tool to use
- Chain multiple tool calls together

Think of agents as "AI that can DO things" beyond just chatting!

## 4.2 Creating Custom Tools

In [ ]:
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain import hub
import random
from datetime import datetime

# Define tool functions
def get_current_time(input_text=""):
    """Get the current time"""
    return datetime.now().strftime("%I:%M %p on %B %d, %Y")

def calculate_math(expression):
    """Calculate a mathematical expression"""
    try:
        # Safe evaluation of basic math
        result = eval(expression, {"__builtins__": {}}, {})
        return f"The result is: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

def get_random_fact(category="general"):
    """Get a random interesting fact"""
    facts = {
        "science": [
            "Water can boil and freeze at the same time in a phenomenon called the triple point.",
            "A day on Venus is longer than its year.",
            "Honey never spoils - archaeologists found 3000-year-old honey that's still edible!"
        ],
        "general": [
            "Bananas are berries, but strawberries aren't!",
            "The shortest war in history lasted 38-45 minutes.",
            "An octopus has three hearts."
        ]
    }
    return random.choice(facts.get(category, facts["general"]))

# Create Tool objects
tools = [
    Tool(
        name="GetTime",
        func=get_current_time,
        description="Gets the current date and time. Use this when asked about the current time or date."
    ),
    Tool(
        name="Calculator",
        func=calculate_math,
        description="Calculates mathematical expressions. Input should be a valid Python math expression like '5 + 3' or '10 * 2'."
    ),
    Tool(
        name="FactGenerator",
        func=get_random_fact,
        description="Provides interesting random facts. Input can be 'science' for science facts or leave empty for general facts."
    )
]

# Initialize LLM
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-35-turbo",
    temperature=0
)

# Get the ReAct prompt
prompt = hub.pull("hwchase17/react")

# Create agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# Test the agent
print("=== Testing Agent with Tools ===")
print("\nQuery 1: What time is it?")
result = agent_executor.invoke({"input": "What time is it?"})
print(f"Final Answer: {result['output']}")

print("\n" + "="*60 + "\n")

print("Query 2: Calculate 15 * 8 + 12")
result = agent_executor.invoke({"input": "Calculate 15 * 8 + 12"})
print(f"Final Answer: {result['output']}")

print("\n" + "="*60 + "\n")

print("Query 3: Tell me a science fact")
result = agent_executor.invoke({"input": "Tell me an interesting science fact"})
print(f"Final Answer: {result['output']}")

## 4.3 Chatbot with Tools and Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

class SmartChatbot:
    """
    Advanced chatbot with tool usage capabilities and memory.
    """
    
    def __init__(self, name, tools_list):
        """
        Initialize smart chatbot with tools.
        
        Args:
            name (str): Chatbot name
            tools_list (list): List of Tool objects
        """
        self.name = name
        self.tools = tools_list
        
        # Initialize LLM
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2024-02-15-preview",
            deployment_name="gpt-35-turbo",
            temperature=0.7
        )
        
        # Setup memory
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        # Create agent
        prompt = hub.pull("hwchase17/react")
        agent = create_react_agent(self.llm, self.tools, prompt)
        
        self.agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            memory=self.memory,
            verbose=False,
            handle_parsing_errors=True
        )
    
    def chat(self, user_input):
        """
        Chat with the smart chatbot.
        
        Args:
            user_input (str): User's message
        
        Returns:
            str: Chatbot's response
        """
        try:
            result = self.agent_executor.invoke({"input": user_input})
            return result['output']
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
    
    def list_tools(self):
        """List available tools"""
        return [tool.name for tool in self.tools]

# Create smart chatbot
smart_bot = SmartChatbot(name="SmartBot", tools_list=tools)

print(f"Available tools: {smart_bot.list_tools()}\n")

# Test multi-turn conversation
print("Turn 1:")
print(f"User: What's the current time?")
print(f"SmartBot: {smart_bot.chat('What is the current time?')}\n")

print("Turn 2:")
print(f"User: Calculate 25 * 4")
print(f"SmartBot: {smart_bot.chat('Calculate 25 * 4')}\n")

print("Turn 3:")
print(f"User: What did you just calculate?")
print(f"SmartBot: {smart_bot.chat('What did you just calculate for me?')}")

# 5. Production-Ready Chatbot Framework

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain import hub
import os
import json
from datetime import datetime

class ProductionChatbot:
    """
    A production-ready chatbot framework with:
    - Conversation memory
    - Tool usage
    - Error handling
    - Logging
    - Customizable personality
    """
    
    def __init__(self, config):
        """
        Initialize chatbot with configuration.
        
        Args:
            config (dict): Configuration dictionary with keys:
                - name: Chatbot name
                - personality: Personality description
                - domain: Area of expertise (optional)
                - memory_window: Number of exchanges to remember
                - temperature: LLM temperature (0-1)
                - tools: List of Tool objects (optional)
        """
        self.config = config
        self.name = config.get('name', 'Assistant')
        self.personality = config.get('personality', 'You are a helpful AI assistant.')
        self.domain = config.get('domain')
        self.memory_window = config.get('memory_window', 5)
        self.temperature = config.get('temperature', 0.7)
        self.tools = config.get('tools', [])
        
        # Conversation log
        self.conversation_log = []
        
        # Initialize LLM
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2024-02-15-preview",
            deployment_name="gpt-35-turbo",
            temperature=self.temperature
        )
        
        # Setup memory
        self.memory = ConversationBufferWindowMemory(
            k=self.memory_window,
            memory_key="chat_history" if self.tools else "history",
            return_messages=bool(self.tools)
        )
        
        # Setup conversation or agent
        if self.tools:
            self._setup_agent()
        else:
            self._setup_conversation()
    
    def _setup_conversation(self):
        """Setup basic conversation chain"""
        system_prompt = f"You are {self.name}. {self.personality}"
        if self.domain:
            system_prompt += f" You specialize in {self.domain}."
        
        template = f"{system_prompt}\n\nConversation history:\n{{history}}\n\nHuman: {{input}}\n{self.name}:"
        
        prompt = PromptTemplate(
            input_variables=["history", "input"],
            template=template
        )
        
        self.chain = ConversationChain(
            llm=self.llm,
            memory=self.memory,
            prompt=prompt,
            verbose=False
        )
    
    def _setup_agent(self):
        """Setup agent with tools"""
        prompt = hub.pull("hwchase17/react")
        agent = create_react_agent(self.llm, self.tools, prompt)
        
        self.agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            memory=self.memory,
            verbose=False,
            handle_parsing_errors=True
        )
    
    def chat(self, user_input):
        """
        Process user input and return response.
        
        Args:
            user_input (str): User's message
        
        Returns:
            dict: Response dictionary with 'response', 'timestamp', 'success'
        """
        timestamp = datetime.now().isoformat()
        
        try:
            # Get response
            if self.tools:
                result = self.agent_executor.invoke({"input": user_input})
                response = result['output']
            else:
                response = self.chain.predict(input=user_input)
            
            # Log conversation
            self.conversation_log.append({
                'timestamp': timestamp,
                'user': user_input,
                'assistant': response,
                'success': True
            })
            
            return {
                'response': response,
                'timestamp': timestamp,
                'success': True
            }
        
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            
            # Log error
            self.conversation_log.append({
                'timestamp': timestamp,
                'user': user_input,
                'assistant': error_msg,
                'success': False,
                'error': str(e)
            })
            
            return {
                'response': "Sorry, I encountered an error processing your request.",
                'timestamp': timestamp,
                'success': False,
                'error': str(e)
            }
    
    def reset(self):
        """Reset conversation memory"""
        self.memory.clear()
        self.conversation_log.append({
            'timestamp': datetime.now().isoformat(),
            'action': 'memory_reset'
        })
    
    def get_conversation_log(self):
        """Get full conversation log"""
        return self.conversation_log
    
    def export_conversation(self, filename):
        """Export conversation log to JSON file"""
        with open(filename, 'w') as f:
            json.dump(self.conversation_log, f, indent=2)
        print(f"Conversation exported to {filename}")
    
    def get_stats(self):
        """Get conversation statistics"""
        total = len([log for log in self.conversation_log if 'user' in log])
        successful = len([log for log in self.conversation_log if log.get('success')])
        failed = total - successful
        
        return {
            'total_exchanges': total,
            'successful': successful,
            'failed': failed,
            'success_rate': f"{(successful/total*100):.1f}%" if total > 0 else "0%"
        }

# Example usage
print("=== Creating Production Chatbot ===")

config = {
    'name': 'HealthBot',
    'personality': 'You are a supportive wellness companion focused on healthy habits.',
    'domain': 'health and wellness',
    'memory_window': 5,
    'temperature': 0.7
}

chatbot = ProductionChatbot(config)

# Test conversation
print("\nTest Conversation:\n")

response1 = chatbot.chat("Hi! I want to improve my sleep.")
print(f"User: Hi! I want to improve my sleep.")
print(f"{chatbot.name}: {response1['response']}\n")

response2 = chatbot.chat("What about exercise?")
print(f"User: What about exercise?")
print(f"{chatbot.name}: {response2['response']}\n")

# Get stats
print("\nConversation Stats:")
print(json.dumps(chatbot.get_stats(), indent=2))

# 6. Best Practices & Tips

##  Key Best Practices

### 1. **Memory Management**
```python
#  Good: Use window memory to control costs
memory = ConversationBufferWindowMemory(k=5)

#  Avoid: Unlimited buffer for long conversations
# memory = ConversationBufferMemory()  # Can get expensive!
```

### 2. **Temperature Selection**
- **0.0-0.3**: Factual tasks, consistent outputs
- **0.7**: Balanced (recommended for most cases)
- **0.9-1.0**: Creative writing, varied responses

### 3. **Error Handling**
```python
try:
    response = chatbot.chat(user_input)
except Exception as e:
    print(f"Error: {e}")
    response = "Sorry, something went wrong."
```

### 4. **Prompt Engineering**
- Be specific and clear
- Provide context and examples
- Define personality and constraints
- Keep system prompts focused

### 5. **API Cost Optimization**
- Limit conversation history
- Use appropriate temperature
- Cache common responses
- Monitor token usage

### 6. **Security**
- Never commit API keys
- Use environment variables
- Validate user inputs
- Implement rate limiting

### 7. **Testing**
```python
test_cases = [
    "Hello, how are you?",
    "What can you help me with?",
    "Tell me about yourself"
]

for test in test_cases:
    response = chatbot.chat(test)
    print(f"Input: {test}")
    print(f"Output: {response}\n")
```

### 8. **Code Organization**
```
project/
├── chatbot.py          # Main chatbot class
├── tools.py            # Custom tool definitions
├── config.py           # Configuration
├── .env                # API keys (gitignored!)
```

### 9. **Common Pitfalls to Avoid**
- Storing sensitive data in prompts
- Not handling API failures
- Unlimited conversation history
- Overly complex system prompts
- Not testing edge cases

### 10. **Deployment Checklist**
- [ ] Environment variables configured
- [ ] Error handling implemented
- [ ] Memory limits set
- [ ] Logging enabled
- [ ] Rate limiting considered
- [ ] API keys secured
- [ ] Testing completed

# 7. Quick Reference Cheat Sheet

## Essential Code Snippets

### Initialize LLM
```python
from langchain_openai import AzureChatOpenAI
import os

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-35-turbo",
    temperature=0.7
)
```

### Create Memory
```python
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=5)
```

### Simple Conversation
```python
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=llm, memory=memory)
response = conversation.predict(input="Hello!")
```

### Custom Prompt
```python
from langchain.prompts import PromptTemplate

template = "You are {role}. {input}"
prompt = PromptTemplate(
    input_variables=["role", "input"],
    template=template
)
```

### Create Tool
```python
from langchain.agents import Tool

tool = Tool(
    name="ToolName",
    func=your_function,
    description="What the tool does"
)
```

### Error Handling
```python
try:
    response = chatbot.chat(user_input)
except Exception as e:
    response = f"Error: {e}"
```